In [ ]:
"""
ABM model: Configuration parameters

@author: hector@bith.net
"""
import sys

In [ ]:
class Config:
    T: int = 1000         # Time (1000)
    N: int = 100          # Number of firms (100)

    eta: float = 0.00001  # ŋ Inverse elasticity: ŋ=1/10000 -> perfect competition, ŋ=1/4 -> high market power

    # firms:                 balance sheet => K = A + L
    firms_K_i0: float = 5.0  # capital
    firms_A_i0: float = 1.0  # assets
    firms_L_i0: float = 4.0  # loans (from bank sector)
    phi: float = 1.1         # Φ capital productivity: constant in this model without R&D
    threshold_bankrupt = 0   # A < threshold_bankrupt and firm will fail
    g: float = 1.1           # cost per unit of capital
    w: float = 0.005         # markdown interest rate (the higher it is, the monopolistic power of bank sector)
    k: float = 1.0           # capital intensity rate K/N
    b: float = 1.0           # parameter of bankruptcy cost (b>0)
    beta: float = 0.02       # β skewness parameter -1 ... 1
    m: float = 0.0           # percentage of K that should be in cash

    # bank sector:                   balance sheet => L = A + D
    bank_sector_A_i0: float = 32.0   # L and D are set inside bank.py
    r_i0: float = 0.02               # initial rate of interest charged to firms by loans
    lambda_param: float = 0.3        # λ, to determine credit allotted for firms L=A/alfa, 0 < λ < 1
    alpha: float = 0.08              # α ratio equity/loan,  Ls=A/α

    # seed used:
    default_seed: int = 20579


    def __str__(self, separator=""):
        description = sys.argv[0]
        for attr in dir(self):
            value = getattr(self, attr)
            if isinstance(value, int) or isinstance(value, float):
                description += f" {attr}={value}{separator}"
        return description

In [ ]:
"""
ABM model

@author: hector@bith.net
"""

In [ ]:
class BankSector:
    def __init__(self, its_model, A_i0=None):
        self.model = its_model
        self.bad_debt = self.profits = self.totalA = self.totalK = 0.0
        self.A = A_i0 if A_i0 else self.model.config.bank_sector_A_i0
        self.L = self.A / self.model.config.alpha
        self.D = self.L - self.A
        if self.D < 0:
            raise ValueError("error bank.D<0 due to " +
                             f"D=L-A={self.model.log.format(self.L)}-{self.model.log.format(self.A)}")
        self.credit_supply = self.determine_new_credit_suppy()
        self.estimate_total_A_K()

    def determine_deposits(self):
        # L = A + D, ----> D = L-A
        return self.L - self.A

    def determine_profits(self):
        # (Equation 34)
        profits_loans = 0.0
        for firm in self.model.firms:
            profits_loans += firm.r * firm.L

        remunerations_of_deposits_and_networth = self.determine_average_interest_rate() * (self.D + self.A)

        return profits_loans - remunerations_of_deposits_and_networth

    def determine_average_interest_rate(self):
        # return 0.01 # TODO
        avg_r = sum(firm.r for firm in self.model.firms) / len(self.model.firms)
        return avg_r if avg_r > self.model.config.r_i0 else self.model.config.r_i0

    def determine_equity(self):
        # (Equation 35) At = At-1 + profits - bad_debt
        return self.A + self.profits - self.bad_debt

    def __str__(self):
        return f"bankSector L={self.model.log.format(self.L)} A={self.model.log.format(self.A)} " + \
            f"D={self.model.log.format(self.D)} cs={self.model.log.format(self.credit_supply)}"

    def determine_step_results(self):
        self.profits = self.determine_profits()
        self.A = self.determine_equity()
        self.L = self.credit_supply
        self.D = self.determine_deposits()
        self.credit_supply = self.determine_new_credit_suppy()

    def determine_firm_capacity_loan(self, firm):
        # (Equation 11 of paper a new approach to business fluctuations)
        return (self.model.config.lambda_param * self.credit_supply * firm.K / self.totalK +
                (1 - self.model.config.lambda_param) * self.credit_supply * firm.A / self.totalA)

    def determine_new_credit_suppy(self):
        return self.A / self.model.config.alpha

    def add_bad_debt(self, amount):
        self.bad_debt += amount

    def estimate_total_A_K(self):
        self.totalA = sum(float(firm.A) for firm in self.model.firms)
        self.totalK = sum(float(firm.K) for firm in self.model.firms)

In [ ]:
"""
ABM model

@author: hector@bith.net
"""
import random

In [ ]:
class Firm:
    def __init__(self, new_id=None, its_model=None):
        if its_model:
            self.id = new_id
            self.model = its_model
            self.failures = 0
            self.debug_info = ""
        self.K = self.model.config.firms_K_i0
        self.A = self.model.config.firms_A_i0
        self.L = self.model.config.firms_L_i0
        self.r = self.model.config.r_i0
        self.gamma = (self.model.config.w / self.model.config.k) + (self.model.config.g * self.r)
        self.phi = self.model.config.phi
        self.pi = self.c = self.Y = self.u = self.gap_of_L = 0.0
        self.desiredK = self.demandL = self.offeredL = self.I = 0.0

    def __str__(self, short: bool = False):
        init = "firm#" if not short else "#"
        if self.failures > 0:
            return f"{init}{self.id}.{self.failures:<2}"
        else:
            return f"{init}{self.id}   "

    def do_step(self):
        self.debug_info = ""
        self.gamma = self.determine_cost_per_unit_of_capital()
        self.desiredK = self.determine_desired_capital()
        self.I = self.determine_investment()
        self.debug_info += f"γ={self.model.log.format(self.gamma)} " + \
                           f"I={self.model.log.format(self.I)}"
        self.demandL = self.determine_demand_loan()
        self.offeredL = self.model.bank_sector.determine_firm_capacity_loan(self)
        self.L += self.determine_new_loan()
        self.r = self.determine_interest_rate()
        self.c = self.determine_marginal_operating_cost()
        self.debug_info += f"c={self.model.log.format(self.c)} r={self.model.log.format(self.r)} "
        self.Y = self.determine_output()
        self.pi = self.determine_profits()
        self.K = self.desiredK
        self.A = self.determine_net_worth()

    def determine_cost_per_unit_of_capital(self):
        # (Before equation 2)  gamma
        return (self.model.config.w / self.model.config.k) + (self.model.config.g * self.r)

    def determine_marginal_operating_cost(self):
        # (Equation 2)
        return self.gamma / self.phi

    def determine_output(self):
        return self.phi * self.desiredK

    def determine_interest_rate(self):
        # (Equation 33)
        # return random.uniform(0.01, 0.05)   #TODO
        return self.model.config.beta * self.L / self.A

    def determine_desired_capital(self):
        # (Equation 22)
        return (1 - self.model.config.eta) ** 2 / (self.model.config.b * self.gamma) - \
            (1 - self.model.config.eta) / (self.model.config.b * self.phi) + \
            self.A / (2 * self.gamma)

    def determine_investment(self):
        # (Below equation 33)
        return self.desiredK - self.K

    def determine_demand_loan(self):
        # (Over equation 33)
        return (1 + self.model.config.m) * self.I - self.pi

    def determine_new_loan(self):
        if self.demandL > self.offeredL:
            self.gap_of_L = (self.demandL - self.offeredL)
            self.K -= self.gap_of_L
            self.debug_info += f" gapL={self.model.log.format(self.gap_of_L)} "
            return self.offeredL
        else:
            self.gap_of_L = 0.0
            return self.demandL

    def determine_u(self):
        # stochastic demand [0,2]
        self.u = random.uniform(0, 2)
        self.debug_info += f"u={self.model.log.format(self.u)} "
        return self.u

    def determine_profits(self):
        # (Equation 24)  , but with Y = phi*K
        return self.determine_u() * (self.model.config.eta + (1 - self.model.config.eta) * self.Y) - \
            self.c * self.Y

    def determine_net_worth(self):
        # (Equation 8)
        return self.A + self.pi

    def check_loses_are_covered_by_m(self):
        return (self.pi < 0) and (self.K * self.model.config.m + self.pi) >= 0

    def is_bankrupted(self):
        return self.A < self.model.config.threshold_bankrupt or self.debug_info.find("failed") > 0

    def set_failed(self):
        self.debug_info += "failed "
        if self.L - self.K > 0:
            self.model.log.debug(f"some error: L={self.L},K={self.K} bankrupted {self}")
        self.model.bank_sector.add_bad_debt(self.K - self.L)
        self.failures += 1
        self.__init__()

    def adjust_capital(self):
        if self.K != (self.A + self.L):
            self.debug_info += f"∆K={self.model.log.format(self.A + self.L - self.K)} "
        return self.A + self.L

In [ ]:
"""
ABM model: It contains the firms and the logic to execute the simulation

@author: hector@bith.net
"""
import random
import statistics

In [ ]:
class Model:
    firms = []
    t: int = 0  # current value of time, t = 0..Model.config.T
    bank_sector: BankSector
    bankruptcies = []

    test = False  # it's true when we are inside a test
    log: Log = None
    statistics: Statistics = None
    config: Config = None
    export_datafile = None
    export_description = None

    def __init__(self, firm_class=Firm, **configuration):
        self.config = Config()
        self.log = Log(self)
        self.firm_class = firm_class
        self.statistics = Statistics(self)
        if configuration:
            self.configure(**configuration)

    def configure(self, **configuration):
        for attribute in configuration:
            if hasattr(self.config, attribute):
                current_value = getattr(self.config, attribute)
                if isinstance(current_value, int):
                    setattr(self.config, attribute, int(configuration[attribute]))
                else:
                    if isinstance(current_value, float):
                        setattr(self.config, attribute, float(configuration[attribute]))
                    else:
                        raise Exception(f"type of config {attribute} not allowed: {type(current_value)}")
            else:
                raise LookupError(f"attribute '{attribute}' in config not found")
        self.initialize_model()

    def initialize_model(self, seed=None,
                         export_datafile=None, export_description=None):

        # what to plot and represent, and in which order
        self.statistics.add(what=Firm, name="K", prepend=" firms   ")
        self.statistics.add(what=Firm, name="A", prepend=" |")
        self.statistics.add(what=Firm, name="L")
        self.statistics.add(what=Firm, name="Y")

        self.statistics.add(what=Firm, name="profits", symbol="π", attr_name="pi")
        self.statistics.add(what=Firm, name="r", function=statistics.mean, repr_function="x̄")
        self.statistics.add(what=Firm, name="Failures", symbol="fail", prepend=" ", attr_name="is_bankrupted",
                            number_type=int)

        self.statistics.add(what=BankSector, name="L", prepend="\n             banks    ")
        self.statistics.add(what=BankSector, name="A", prepend=" | ")
        self.statistics.add(what=BankSector, name="D", prepend=" ")

        self.statistics.add(what=BankSector, name="profits", symbol="π", prepend=" ", plot=False, attr_name="profits")
        self.statistics.add(what=BankSector, name="bad debt",
                            symbol="bd", prepend=" ", plot=False, attr_name="bad_debt")
        self.statistics.add(what=BankSector, name="credit supply", symbol="cs", prepend=" ", plot=False,
                            attr_name="credit_supply")

        self.config.__init__()
        random.seed(seed if seed else self.config.default_seed)
        self.export_datafile = export_datafile
        self.export_description = str(self.config) if export_description is None else export_description
        self.firms = []
        for i in range(self.config.N):
            self.firms.append(self.firm_class(new_id=i, its_model=self))
        self.bank_sector = BankSector(self)
        if not self.test:
            self.statistics.debug_firms(before_start=True)
            self.log.info(self.statistics.current_status_save(), before_start=True)

    def do_step(self):
        self.bank_sector.bad_debt = 0.0
        self.bank_sector.determine_new_credit_suppy()
        for firm in self.firms:
            firm.do_step()
        self.bank_sector.determine_step_results()
        self.statistics.debug_firms()
        self.remove_failed_firms()
        self.log.info(self.statistics.current_status_save())

    def finish_model(self):
        if not self.test:
            self.statistics.export_data(self.export_datafile, self.export_description)
            self.statistics.plot()
        self.log.info(f"finish: model T={self.config.T} N={self.config.N}")
        self.statistics.export_data(export_datafile=self.export_datafile, export_description=self.export_description)

    def run(self, export_datafile=None):
        self.initialize_model(export_datafile=export_datafile)
        for self.t in range(self.config.T):
            self.do_step()
        self.finish_model()

    def remove_failed_firms(self):
        for firm in self.firms:
            if firm.is_bankrupted():
                firm.set_failed()
        self.bank_sector.estimate_total_A_K()

In [ ]:
"""
ABM model auxiliary file: logging facilities
@author: hector@bith.net
"""
import logging
import numpy as np
import sys

In [ ]:
class Log:
    """
    The class acts as a logger and helpers to represent the data and evol from the Model.
    """
    logger = logging.getLogger("model")
    model = None
    logLevel = "ERROR"

    def __init__(self, its_model):
        self.model = its_model

    @staticmethod
    def format(number):
        if isinstance(number, int) or isinstance(number, np.int32):
            result = f"{number:3}"
        else:
            result = f"{number:5.2f}"
            while len(result) > 5 and result[-1] in "0":
                result = result[:-1]
            while len(result) > 5 and result.find('.') > 0:
                result = result[:-1]
        return result if result[-1]!='.' else f' {result[:-1]}'

    @staticmethod
    def get_level(option):
        try:
            return getattr(logging, option.upper())
        except AttributeError:
            logging.error(f" '--log' must contain a valid logging level and {option.upper()} is not.")
            sys.exit(-1)

    def debug(self, text, before_start=False):
        if not self.model.test:
            self.logger.debug(f"{self.__format_t__(before_start)} {text}")

    def info(self, text, before_start=False):
        if not self.model.test:
            self.logger.info(f" {self.__format_t__(before_start)} {text}")

    def error(self, text, before_start=False):
        if not self.model.test:
            self.logger.error(f"{self.__format_t__(before_start)} {text}")

    def __format_t__(self, before_start=False):
        return "     " if before_start else f"t={self.model.t:03}"

    def define_log(self, log: str, logfile: str = ''):
        formatter = logging.Formatter('%(levelname)s %(message)s')
        self.logLevel = Log.get_level(log.upper())
        self.logger.setLevel(self.logLevel)
        if logfile:
            if not logfile.startswith(self.model.statistics.OUTPUT_DIRECTORY):
                logfile = f"{self.model.statistics.OUTPUT_DIRECTORY}/{logfile}"
            fh = logging.FileHandler(logfile, 'a', 'utf-8')
            fh.setLevel(self.logLevel)
            fh.setFormatter(formatter)
            self.logger.addHandler(fh)
        else:
            ch = logging.StreamHandler()
            ch.setLevel(self.logLevel)
            ch.setFormatter(formatter)
            self.logger.addHandler(ch)

In [ ]:
"""
ABM model auxiliary file: to have statistics and plot
@author:  hector@bith.net
"""

In [ ]:
class Statistics:
    OUTPUT_DIRECTORY = "output"
    enable_plot = False

    # This time the idea is to use pandas to store the statistics
    def __init__(self, its_model):
        self.model = its_model
        self.data = {}
        import os
        if not os.path.isdir(self.OUTPUT_DIRECTORY):
            os.mkdir(self.OUTPUT_DIRECTORY)

    def debug_firms(self, before_start=False):
        for firm in self.model.firms:
            self.debug_firm(firm, before_start=before_start)

    def debug_firm(self, firm, before_start=False):
        text = f"{firm.__str__()} K={Log.format(firm.K)}"
        text += f" | A={Log.format(firm.A)} L={Log.format(firm.L)}"
        if not before_start:
            # text += f", Y={Log.format(firm.Y)}"
            text += f" π={Log.format(firm.pi)}"
            # text += f" γ={Log.format(firm.gamma)}"
            text += f" dK={Log.format(firm.desiredK)}"
            text += f" dL/oL={Log.format(firm.demandL)}/{Log.format(firm.offeredL)}"
            text += " bankrupted" if firm.is_bankrupted() else ""
            text += " " + firm.debug_info
        self.model.log.debug(text, before_start)

    def current_status_save(self):
        # it returns also a string with the status
        result = ""
        for item in self.data:
            result += self.data[item].store_statistics()
        return result

    def add(self, what, name, prepend="", symbol=None, attr_name=None, number_type=float, function=sum,
            repr_function="Σ", plot=True, log=False):
        if not attr_name:
            attr_name = name
        if not symbol:
            symbol = name.replace(" ", "_")
            if len(symbol) != len(name):
                symbol = symbol.lower()
        if not callable(function):
            raise TypeError("function parameter should be a callable type")
        if what == BankSector:
            self.data["bank"+name] = StatsBankSector(self.model, number_type, name, symbol, prepend=prepend, plot=plot,
                                                     attr_name=attr_name, log=log)
        else:
            self.data["firm"+name] = StatsFirms(self.model, number_type, name, symbol, prepend=prepend,
                                                function=function, repr_function=repr_function,
                                                plot=plot, attr_name=attr_name, log=log)

    @staticmethod
    def get_export_path(filename):
        if not filename.startswith(Statistics.OUTPUT_DIRECTORY):
            filename = f"{Statistics.OUTPUT_DIRECTORY}/{filename}"
        return filename if filename.endswith('.txt') else f"{filename}.txt"

    def export_data(self, export_datafile=None, export_description=None):
        if export_datafile:
            with open(Statistics.get_export_path(export_datafile), 'w', encoding="utf-8") as savefile:
                if export_description:
                    savefile.write(f"# {export_description}\n")
                else:
                    savefile.write(f"# {__name__} T={self.model.config.T} N={self.model.config.N}\n")
                header = "# t"
                for item in self.data:
                    header += "\t" + self.data[item].__str__()
                savefile.write(header + "\n")
                for i in range(self.model.config.T):
                    line = f"{i:3}"
                    for item in self.data:
                        line += "\t" + self.data[item][i]
                    savefile.write(line + "\n")

    def plot(self):
        if self.enable_plot:
            for item in self.data:
                self.data[item].plot()

In [ ]:
def mean(data):
    """ returns the mean of an array"""
    result = 0
    for i,x in enumerate(data):
        result += x
    return result/i

In [ ]:
"""
ABM model executer, to run interactively the model
@author: hector@bith.net
"""
import typer

In [ ]:
def run_interactive(getconfig: bool = typer.Option(False, help="Get current configuration"),
                    setconfig: List[str] = typer.Argument(None, help="Change config value (i.e. alpha=3.1"),

                    log: str = typer.Option('ERROR', help="Log level messages (ERROR,DEBUG,INFO...)"),
                    logfile: str = typer.Option(None, help="File to send logs to"),
                    save: str = typer.Option(None, help="Saves the output of this execution"),
                    plot: bool = typer.Option(False, help="Saves the plots"),
                    n: int = typer.Option(Config.N, help="Number of firms"),
                    t: int = typer.Option(Config.T, help="Time repetitions")):
    global model
    if t != model.config.T:
        model.config.T = t
    if n != model.config.N:
        model.config.N = n
    model.log.define_log(log, logfile)
    if plot:
        model.statistics.enable_plot = True
    if getconfig:
        print(model.config.__str__(separator="\n"))
        raise typer.Exit()
    manage_config_values(setconfig)
    run(save)

In [ ]:
def run(save=None):
    global model
    model.run(export_datafile=save)

In [ ]:
def manage_config_values(config_list):
    for item in config_list:
        try:
            name_config, value_config = item.split("=")
        except ValueError:
            model.log.error("A config value should be passed as name=value", before_start=True)
            raise typer.Exit(-1)
        try:
            getattr(model.config, name_config)
        except AttributeError:
            model.log.error(f"Configuration has no {name_config}", before_start=True)
            raise typer.Exit(-1)
        try:
            setattr(model.config, name_config, float(value_config))
        except ValueError:
            model.log.error(f"Value given {value_config} is not valid", before_start=True)
            raise typer.Exit(-1)

In [ ]:
def is_notebook():
    try:
        __IPYTHON__
        return get_ipython().__class__.__name__ != "SpyderShell"
    except NameError:
        return False

In [ ]:
model = Model()
if is_notebook():
    # if we are running in a Notebook:
    run()
else:
    # if we are running interactively:
    if __name__ == "__main__":
        typer.run(run_interactive)